In [1]:
# Importing Libraries
from ampere_utils.plot import polar_plot, configure_polar_plot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation, FFMpegWriter
import xarray as xr
import datetime as dt

In [2]:
# Loading Data:
filepath = r"C:\Users\Furio\Documents\University\Master's\Internship\Data" # Change Appropriately
Ampere_file_location = filepath + r"\AMPERE\grd\2015"
Ampere_file = Ampere_file_location + r"\ampere.20150101.0000.86400.120.north.grd.ncdf"
Ampere_data = xr.load_dataset(Ampere_file)
Ampere_data

<xarray.Dataset> Size: 180MB
Dimensions:       (nRec: 720, nObs: 1200, vComp: 3)
Dimensions without coordinates: nRec, nObs, vComp
Data variables: (12/34)
    npnt          (nRec) int16 1kB 1200 1200 1200 1200 ... 1200 1200 1200 1200
    year          (nRec) int16 1kB 2015 2015 2015 2015 ... 2015 2015 2015 2015
    doy           (nRec) int16 1kB 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    time          (nRec) float32 3kB 0.0 0.03333 0.06667 ... 23.9 23.93 23.97
    avgint        (nRec) int16 1kB 600 600 600 600 600 ... 600 600 600 600 600
    kmax          (nRec) int16 1kB 60 60 60 60 60 60 60 ... 60 60 60 60 60 60 60
    ...            ...
    del_db_geo    (nRec, nObs, vComp) float64 21MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    del_jPar      (nRec, nObs) float64 7MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    del_db_Th_Th  (nRec, nObs) float64 7MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    del_db_Ph_Th  (nRec, nObs) float64 7MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    del_db_Th_Ph  (nRec, nObs) float64 7MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    del_db_Ph_Ph  (nRec, nObs) float64 7MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    title:        Ampere data for 2015-01-01 00:00 - 2015-01-02 00:00.
    description:  AMPERE GRD data (fitted magnetic pertubation data).
    created:      2023-03-22T17:04:52-04:00

In [3]:
Ampere = xr.where(Ampere_data['jPar'] >= 9990, np.nan, Ampere_data['jPar'])
Ampere = Ampere.values.reshape(720, 24, 50)

In [10]:
mlt = np.arange(24)
clat = np.arange(0, 50)

# Figure
fig, ax = plt.subplots(figsize=(15, 8), subplot_kw={'projection': 'polar'})
cbar_ax = fig.add_axes([0.92, 0.15, 0.03, 0.7])

# Intial Frame
initial_frame = 0
ampere_frame = Ampere[initial_frame, :, :]
plot_obj = polar_plot(mlt, clat, ampere_frame, hemisphere="north", ax=ax)
norm1 = mcolors.TwoSlopeNorm(vmin=-0.4, vcenter=0, vmax=0.4)
plot_obj.set_norm(norm1)
ax.set_title(f"Ampere Data (Northern Hemisphere) - Frame {initial_frame}", pad = 0.5)
cbar = fig.colorbar(plot_obj, cax=cbar_ax, extend="both")
cbar.set_label("Ampere (µA/m²)")


def update(frame):
    print(f"Processing frame {frame + 1} of {total_frames}")
    ax.clear() 
    ampere_frame = Ampere[frame, :, :]
    plot_obj = polar_plot(mlt, clat, ampere_frame, hemisphere="north", ax=ax)
    norm1 = mcolors.TwoSlopeNorm(vmin=-0.4, vcenter=0, vmax=0.4)
    plot_obj.set_norm(norm1)
    ax.set_title(f"Ampere Data (Northern Hemisphere) - Frame {frame}", pad = 0.5)
    cbar.update_normal(plot_obj)
    return [plot_obj]

total_frames = Ampere.shape[0]
selected_frames = range(0, total_frames, 200)

anim = FuncAnimation(fig, update, frames=selected_frames, interval=500, blit=False)
output_file = "Simple_Video.mp4"
writer = FFMpegWriter(fps=2)
anim.save(output_file, writer=writer)
print(f"Animation saved as {output_file}")

plt.close(fig)

Processing frame 1 of 720
Processing frame 1 of 720
Processing frame 201 of 720
Processing frame 401 of 720
Processing frame 601 of 720
Animation saved as Simple_Video.mp4
